In [1]:
%load_ext autoreload
%autoreload 2
%env ANYWIDGET_HMR=1

env: ANYWIDGET_HMR=1


In [4]:
def generate_images(count=10):
    images = []
    for i in range(count):
        url = f"https://picsum.photos/seed/{i + 1}/300/200"
        images.append({
            "url": url,
            "title": f"Image {i+1}"
        })
    return images

In [10]:
from ipygallery import ImagePicker
images = generate_images(150)
w = ImagePicker(images=images, height="300px", width="100%")
display(w)


ImagePicker(height='300px', images=[{'url': 'https://picsum.photos/seed/1/300/200', 'title': 'Image 1'}, {'url…